In [3]:
!pip install pandas

  Using cached pandas-1.4.2-cp39-cp39-macosx_11_0_arm64.whl (10.1 MB)


In [5]:
import pandas as pd
import requests

In [7]:
rq = requests.get(
    'https://api.pushshift.io/reddit/search/submission',
    params={
        'subreddit': 'datascience',
        'sort': 'asc',
        'sort_type': 'created_utc',
        'after': 1546297200,
        'before': 1651059776,
        'size': 500
    }
)
rq.ok

True

In [12]:
data = rq.json()['data']
lst = [[post['title'], post['selftext'], post['full_link'], post['num_comments'], post['score'], post['retrieved_on']] for post in data]

In [14]:
df = pd.DataFrame(data, columns=['title', 'selftext', 'full_link', 'num_comments', 'score', 'retrieved_on'])

In [15]:
df.head()

,title,selftext,full_link,num_comments,score,retrieved_on
0,What kind of jobs should I shoot for just to g...,So I've been going through some online boot ca...,https://www.reddit.com/r/datascience/comments/...,1,1,1546306578
1,A collection of Bash scripts to install data s...,,https://www.reddit.com/r/datascience/comments/...,0,1,1546315392
2,6 Major Benefits of Open Data,,https://www.reddit.com/r/datascience/comments/...,0,1,1546321019
3,Hey Google! When did I ask you to read my Purc...,,https://www.reddit.com/r/datascience/comments/...,0,1,1546330166
4,An Idiot ’ s guide to Support vector machines ...,,https://www.reddit.com/r/datascience/comments/...,6,1,1546339598


In [37]:
from datetime import datetime
import os

jan_2019_timestamp = 1546297200
import time

def fetch_all_data(after_timestamp=jan_2019_timestamp):
    fname = f"reddit-{after_timestamp}.csv"
    print(f"Running for {after_timestamp}")
    params = {
        'subreddit': 'datascience',
        'sort': 'asc',
        'sort_type': 'created_utc',
        'after': after_timestamp,
        'before': 1651059776,
        'size': 500
    }
    req = requests.get('https://api.pushshift.io/reddit/search/submission', params=params)
    if not req.ok:
        if req.status_code == 429:
            time.sleep(1)
            print("Retrying...")
            return fetch_all_data(after_timestamp)
    data = req.json()['data']
    lst = [[post['title'], post['selftext'] if 'selftext' in 'post' else None, post['full_link'], post['num_comments'], post['score'], post['retrieved_on']] for post in data]
    df = pd.DataFrame(data, columns=['title', 'selftext', 'full_link', 'num_comments', 'score', 'retrieved_on'])
    prev_timestamp = df['retrieved_on'].max()
    if not prev_timestamp:
        print("Finished!")
        return
    df.to_csv(fname)
    print(f"Finished with {datetime.fromtimestamp(after_timestamp)}, starting for {datetime.fromtimestamp(prev_timestamp)}")
    
    return fetch_all_data(prev_timestamp)
    

In [ ]:
fetch_all_data()

In [45]:
import glob
joined_files = os.path.join(".", "reddit-*.csv")
  
# A list of all joined files is returned
joined_list = glob.glob(joined_files)
  
# Finally, the files are joined
df = pd.concat(map(pd.read_csv, joined_list), ignore_index=True)
df['relevance'] = -1
df.to_csv('full_dataset.csv')

In [46]:

import ray
ray.init()

RayContext(dashboard_url='', python_version='3.9.10', ray_version='1.12.0', ray_commit='f18fc31c7562990955556899090f8e8656b48d2d', address_info={'node_ip_address': '127.0.0.1', 'raylet_ip_address': '127.0.0.1', 'redis_address': None, 'object_store_address': '/tmp/ray/session_2022-04-27_16-01-06_600500_59654/sockets/plasma_store', 'raylet_socket_name': '/tmp/ray/session_2022-04-27_16-01-06_600500_59654/sockets/raylet', 'webui_url': '', 'session_dir': '/tmp/ray/session_2022-04-27_16-01-06_600500_59654', 'metrics_export_port': 61043, 'gcs_address': '127.0.0.1:61365', 'address': '127.0.0.1:61365', 'node_id': '1d636d13a996f330004f13d7ed0282950021862ebaca5e1c454f34de'})

In [49]:
!pip install pyarrow
!pip install ray[data]

zsh:1: no matches found: ray[data]


In [50]:
ds = ray.data.from_pandas(df)

In [52]:
ds.write_csv('training-data.csv')

Write Progress: 100%|███████████████████████████████████████| 1/1 [00:00<00:00,  1.72it/s]
